Search algorithms

In [37]:
from gx_utils import *

In [38]:
import logging
from random import seed, choice
from typing import Callable

logging.basicConfig(format="%(message)s", level=logging.INFO)

In [39]:
import numpy as np

In [40]:
class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [48]:
def find_empty_space(board: np.ndarray):
    t = np.where(board == 0)
    return np.array([t[0][0], t[1][0]])

def is_valid(board: np.ndarray, action):
    return all(0 <= (find_empty_space(board) + action)[i] < board.shape[i] for i in [0, 1])

def possible_actions(state: State):
    return (m for m in MOVES if is_valid(state._data, m))

In [ ]:
def result(state, action):
    board = state.copy_data()
    space = find_empty_space(board)
    pos = space + action
    board[space[0], space[1]] = board[pos[0], pos[1]]
    board[pos[0], pos[1]] = 0
    return State(board)

In [41]:
def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
):
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0

    while state is not None and not goal_test(state):
        for a in possible_actions(state):
            new_state = result(state, a)
            cost = unit_cost(a)
            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    logging.info(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    return list(reversed(path))

In [42]:
import random

In [43]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [50]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

def greedy(N):
    goal=set(range(N))
    covered=set()
    print("PROVA")
    solution=list()
    all_lists=sorted(problem(N,seed=42),key=lambda l:len(l))
    while goal != covered:
        x=all_lists.pop(0)
        if not set(x) < covered: #sottoinsieme stretto
            solution.append(x)
            covered |= set(x) #unione fra i due set e prende gli elementi distinti
    print(f"Greedy solution: {solution}")

In [ ]:
greedy(1000)

PROVA
Greedy solution: [[8192, 8200, 8201, 10, 12, 14, 8208, 8211, 8220, 8222, 8223, 8225, 34, 35, 36, 8230, 8251, 8254, 66, 8262, 8267, 76, 79, 8272, 82, 8276, 8283, 92, 94, 95, 96, 8287, 97, 99, 106, 108, 109, 111, 116, 117, 8312, 8313, 123, 126, 8320, 129, 131, 132, 133, 135, 8328, 137, 8329, 139, 8332, 8331, 143, 148, 152, 154, 157, 160, 164, 8357, 169, 8367, 8370, 8371, 180, 181, 8374, 8377, 185, 189, 8383, 8398, 210, 8402, 8405, 218, 221, 8420, 8422, 233, 8427, 236, 245, 247, 248, 8441, 8442, 8444, 8446, 257, 266, 8465, 273, 8470, 8471, 283, 288, 295, 8488, 8491, 8492, 8495, 8496, 303, 304, 308, 8500, 310, 313, 315, 8513, 8516, 324, 326, 8519, 329, 8523, 8524, 334, 339, 8533, 342, 343, 341, 8545, 8546, 354, 355, 357, 358, 359, 362, 8560, 8562, 8564, 380, 8576, 386, 388, 389, 392, 8585, 394, 8589, 8591, 8592, 8594, 8595, 405, 8602, 413, 8608, 8610, 421, 429, 8621, 430, 433, 8626, 435, 436, 8633, 8634, 8635, 445, 8645, 457, 8650, 8662, 8663, 8664, 8666, 476, 479, 491, 8684, 8685, 4

: 

: 